<a href="https://colab.research.google.com/github/dm-fedorov/pandas_basic/blob/master/%D0%BF%D0%BE%D0%B4%D1%80%D0%BE%D0%B1%D0%BD%D0%B5%D0%B5%20%D0%B8%D0%B7%D1%83%D1%87%D0%B0%D0%B5%D0%BC%20pandas/12_%D0%93%D1%80%D1%83%D0%BF%D0%BF%D0%B8%D1%80%D0%BE%D0%B2%D0%BA%D0%B0%20%D0%B8%20%D0%B0%D0%B3%D1%80%D0%B5%D0%B3%D0%B8%D1%80%D0%BE%D0%B2%D0%B0%D0%BD%D0%B8%D0%B5.ipynb" target="_blank"><img align="left" src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open in Colab" title="Open and Execute in Google Colaboratory"></a>

In [ ]:
# импортируем библиотеки numpy и pandas
import numpy as np
import pandas as pd

# импортируем библиотеку datetime для работы с датами
import datetime
from datetime import datetime, date

# импортируем библиотеку matplotlib для построения графиков
import matplotlib.pyplot as plt

### Схема: разделение, применение, объединение

![](pic/pandas01.png)

Разделение выполняется с помощью метода `.groupby()` объекта `Series` или `DataFrame`.

После разделения данных для каждой группы можно выполнить:

- **агрегацию**: вычислить итоговую статистику
- **преобразование**: выполнить вычисление по группам
- **фильтацию**: удаление групп данных.

Объединие происходит автоматически.

[Оригинальная статья о методе](https://www.jstatsoft.org/article/view/v040i01/v40i01.pdf)

In [ ]:
url = "https://raw.githubusercontent.com/dm-fedorov/pandas_basic/master/%D0%BF%D0%BE%D0%B4%D1%80%D0%BE%D0%B1%D0%BD%D0%B5%D0%B5%20%D0%B8%D0%B7%D1%83%D1%87%D0%B0%D0%B5%D0%BC%20pandas/Data/orders.csv"

In [ ]:
# загружаем данные
orders = pd.read_csv(url)
orders.head()

In [ ]:
orders.sample(5)

## Разделение данных

Сначала рассмотрим группировку на основе столбцов, затем разберем свойства группировки.

### Группировка по значениям отдельного столбца

In [ ]:
# группировка данных по столбцу/переменной 
# возвращает объект DataFrameGroupBy
grouped_by_orders = orders.groupby('customer_id') # sort=True
grouped_by_orders

Получили промежуточное описание группировки, помогает выявить ошибки перед выполнением и имеет ряд свойств.

In [ ]:
# получаем информацию о количестве групп, 
# которые будут созданы
grouped_by_orders.ngroups

In [ ]:
# что представляют из себя найденные группы? много строк!
#grouped_by_orders.groups

### Просмотр результатов группировки

In [ ]:
# вспомогательная функция, печатающая содержимое групп
def print_groups(group_object):
    # итерируем по всем группам, печатая название группы 
    # и первые пять наблюдений в группе
    for name, group in group_object:
        print(name)
        print(group[:5])

In [ ]:
# смотрим содержимое созданных групп. много строк! 
#print_groups(grouped_by_orders)

In [ ]:
# получаем информацию о количестве элементов
# в каждой группе
grouped_by_orders.size()

In [ ]:
# получаем информацию о количестве элементов
# в каждом столбце каждой группы
grouped_by_orders.count()

In [ ]:
# получаем данные конкретной группы
grouped_by_orders.get_group('AA-10315')[:5]

In [ ]:
# извлекаем первые три строки в каждой группе
grouped_by_orders.head(3)

In [ ]:
# извлекаем вторую строку каждой группы
grouped_by_orders.nth(1)

In [ ]:
# получаем описательные статистики по каждой группе
grouped_by_orders.describe()

### Группировка по нескольким столбцам

In [ ]:
# группируем по значениям двух столбцов 
by_orders = orders.groupby(['customer_id', 'order_date'])
# много строк
# print_groups(by_orders) 

### Группировка по уровням индекса

Изменим форму данных, создав иерархический индекс, у которого уровнями будут столбцы sensor и axis:

In [ ]:
# создаем копию данных и заново индексируем ее
orders_copy = orders.copy()
orders_copy = orders.set_index(['customer_id', 'order_date'])
orders_copy

In [ ]:
# группируем по первому уровню индекса
#print_groups(orders_copy.groupby(level=0))

In [ ]:
# группируем по нескольким уровням индекса
#print_groups(orders_copy.groupby(level=['customer_id', 'order_date']))

### Применение агрегирующих функций к группам

Агрегирующие функции можно применить к каждой группе с помощью метода `.agg()` объекта `GroupBy`. 

Параметр - ссылка на функцию, которая будет применяться к каждой группе. 

В случае `DataFrame` эта функция будет применяться к каждому столбцу.

In [ ]:
# вычисляем среднее значение
by_orders = orders_copy.groupby(level=['customer_id', 'order_date'])
by_orders['sales'].agg(np.mean)

Вместо индекса, совпадающего с индексом исходного объекта, создаем числовой индекс и переносим уровни исходного индекса в столбцы:

In [ ]:
orders.groupby(['customer_id', 'order_date'], as_index=False)['sales'].agg(np.mean)

In [ ]:
# можно просто применить агрегирующую функцию к группе
by_orders.sales.mean()

[Список всех доступных агрегирующих функций](https://pandas.pydata.org/pandas-docs/stable/reference/groupby.html#computations-descriptive-stats)

[Официальная документация по group by](https://pandas.pydata.org/pandas-docs/stable/user_guide/groupby.html)

In [ ]:
# применяем сразу несколько агрегирующих функций
by_orders.sales.agg([np.sum, np.std])

In [ ]:
# применяем к каждому столбцу свою функцию
by_orders.agg({'sales' : np.mean,
               'ship_mode': len})

### Преобразование групп данных

Метод .transform() объекта GroupBy применяет функцию к каждому значению объекта DataFrame и возвращает другой объект DataFrame.

In [ ]:
transform_data = pd.DataFrame({'Label': ['A', 'C', 'B', 'A', 'C'],
                               'Values': [0, 1, 2, 3, 4],
                               'Values2': [5, 6, 7, 8, 9],
                               'Other': ['foo', 'bar', 'baz', 'fiz', 'buz']},
                               index = list('VWXYZ'))
transform_data

In [ ]:
# сгруппируем данные по столбцу Label
grouped_by_label = transform_data.groupby('Label')
print_groups(grouped_by_label)

In [ ]:
# добавляем 10 ко всем значениям во всех столбцах
grouped_by_label.transform(lambda x: x + 10)

Столбцы, которые имеют строковые значения, исключены из результата.

### Заполнение пропущенных значений групповым средним

Часто необходимо заменить пропущенные данные в каждой группе групповым средним (на основе непропущенных значений). 

In [ ]:
df = pd.DataFrame({ 'Label': list("ABABAB"),
                    'Values': [10, 20, 11, np.nan, 12, 22]})
df

In [ ]:
grouped = df.groupby('Label')
print_groups(grouped)

In [ ]:
# вычисляем среднее для каждой группы
grouped.mean()

In [ ]:
# используем метод .transform(), чтобы заполнить
# значения NaN групповым средним
filled_NaNs = grouped.transform(lambda x: x.fillna(x.mean()))
filled_NaNs

### Исключение групп из процедуры агрегирования

С помощью метода `.filter()` можно выборочно удалить группы данных из обработки. 

В этот метод передают функцию, с помощью которой по каждой группе можно принять решение о включении данной группы в итоговый результат. 

In [ ]:
df = pd.DataFrame({'Label': list('AABCCC'),
                   'Values': [1, 2, 3, 4, np.nan, 8]})
df

In [ ]:
# удаляем группы с одним непропущенным значением и меньше
f = lambda x: x.Values.count() > 1
df.groupby('Label').filter(f)

In [ ]:
# удаляем группы, в которых есть пропуски
f = lambda x: x.Values.isnull().sum() == 0
df.groupby('Label').filter(f)

In [ ]:
# отбираем группы со средним 2.0 и выше 
grouped = df.groupby('Label')
group_mean = grouped.mean().mean()
f = lambda x: abs(x.Values.mean() - group_mean) > 2.0
df.groupby('Label').filter(f)